In [19]:
import copy

import numpy as np
import pandas as pd
from itertools import combinations

from sklearn.model_selection import KFold
from skopt.space import Real, Categorical, Integer

import sys
sys.path.append('G:\내 드라이브\대학\대외\2023\연구원\연구과제\Ensemble-Kernel')

from Ensemble_Kernel_New import split_data,prepare_response_variable, c_o, nom, new_kernel, train_predict_fastKernelSurvivalSVM_bayesian, train_predict_fastKernelSurvivalSVM_clinical_bayesian, one_hot_encode_columns, remove_high_vif_columns, convert_categorical_to_boolean, compare_kernels_with_abs_and_no_abs, train_predict_fastKernelSurvivalSVM_bayesian_no_abs, train_predict_fastKernelSurvivalSVM, train_predict_fastKernelSurvivalSVM_AFT, new_kernel_AFT, train_predict_fastKernelSurvivalSVM_bayesian_AFT, train_predict_fastKernelSurvivalSVM_clinical, train_predict_fastKernelSurvivalSVM_clinical_bayesian, C_index_fastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM, FastKernelSurvivalSVM
from sklearn.model_selection import ShuffleSplit, GridSearchCV

### 전처리

In [7]:
data = pd.read_csv("lung(mice).csv")

df = copy.deepcopy(data)

column_mapping = {'age' : 'Age',
                  'sex' : 'Sex',
                  'time' : 'OS',
                  'status' : 'Status'}
df_rename = df.rename(columns = column_mapping)

df_drop = df_rename.drop(['Unnamed: 0','inst'], axis=1).dropna()

columns_to_convert = ['Sex']
df_drop[columns_to_convert] = df_drop[columns_to_convert].astype('category')

df_onehot, categorical_columns = one_hot_encode_columns(df_drop)

columns_to_convert = ['Sex_2']
df_onehot[columns_to_convert] = df_onehot[columns_to_convert].astype('category')

df_onehot = df_onehot.rename(columns = {'Sex_2' : 'Sex'})

df_onehot

,OS,Status,Age,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss,Sex
0,306,1,74,1,90,100,1175,37,0
1,455,1,68,0,90,90,1225,15,0
2,1010,0,56,0,90,90,1500,15,0
3,210,1,57,1,90,60,1150,11,0
4,883,1,60,0,100,90,1025,0,0
...,...,...,...,...,...,...,...,...,...
223,188,0,77,1,80,60,1225,3,0
224,191,0,39,0,90,90,2350,-5,0
225,105,0,75,2,60,70,1025,5,1
226,174,0,66,1,90,100,1075,1,0


In [48]:
df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   OS         228 non-null    int64   
 1   Status     228 non-null    int64   
 2   Age        228 non-null    int64   
 3   ph.ecog    228 non-null    int64   
 4   ph.karno   228 non-null    int64   
 5   pat.karno  228 non-null    int64   
 6   meal.cal   228 non-null    int64   
 7   wt.loss    228 non-null    int64   
 8   Sex        228 non-null    category
dtypes: category(1), int64(8)
memory usage: 14.7 KB


Kfold settion

In [8]:
cv = KFold(n_splits=5, shuffle=True, random_state=36)

param_grid = {'alpha': 2. ** np.arange(-12, 13, 2)}
param_space = {'alpha': Real(1e-6, 1e+6, 'log-uniform'),}

random state 숫자 100개 선택

In [9]:
import random

with open("random_state_100.txt", "r") as file:
    random_state=file.read()
    
random_state=random_state.split("\n")
random_state=[int(x) for x in random_state if x]

### Cox - cross validation

In [10]:
from kernel_function import ensemble_AFT_kernel, ensemble_cox_kernel, c_index_kernel_type

Best c-index를 도출하는 변수 뽑기

In [11]:
remaining_variable_cox=pd.DataFrame()

remaining_variable_cox['variable']=[]

In [14]:
for i in range(100):
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    #os, status 분리
    drop = x_train[['Sex', 'Age', 'OS','Status']]
    x_train_drop = x_train.drop(columns=['Sex', 'Age', 'OS','Status'])

    # 데이터프레임의 열 이름을 리스트로 저장
    columns = x_train_drop.columns

    column_groups = []

    for i in range(len(columns)):
        # i+1개씩 선택한 모든 열 조합을 생성
        all_column_combinations = list(combinations(columns, i+1))

        # 각 조합에 대한 데이터프레임을 생성하고 출력
        for column_combination in all_column_combinations:
            selected_columns = list(column_combination)
            sub_train1 = x_train_drop[selected_columns]
            sub_train2 = pd.concat([sub_train1, drop], axis=1)
            column_groups.append(sub_train2)
    
    num_groups = 3
    best_cindex = []

    for i in range(len(column_groups)):
        x_groups = []
        train = column_groups[i].sample(frac=1).reset_index(drop=True)

        # train data를 그룹으로 나누기
        for i in range(num_groups):
            censored = train[train['Status'] == 0]  # 0인 행만 추출
            uncensored = train[train['Status'] == 1]  # 1인 행만 추출

            group_size1 = len(censored) // num_groups
            group_size2 = len(uncensored) // num_groups

            if i < num_groups - 1:
                #censoring 비율 맞추기
                group1 = censored.iloc[i * group_size1:(i + 1) * group_size1]
                group2 = uncensored.iloc[i * group_size2:(i + 1) * group_size2]

                group = pd.concat([group1, group2], ignore_index=True)
            else:
                group1 = censored.iloc[i * group_size1:]
                group2 = uncensored.iloc[i * group_size2:]

                group = pd.concat([group1, group2], ignore_index=True)
    
            x_groups.append(group)

        cindex = []

        for i in range(len(x_groups)):

            temp = []
            for j in range(len(x_groups)):
                if i != j:
                    temp.append(x_groups[j])
            #train = train data(등분한 것 중 1개를 제외한 나머지)
            train=pd.concat(temp)

            train = train.reset_index(drop=True)

            validation = x_groups[i]
            
            train_target=train[['Status','OS']]
            test_target=validation[['Status','OS']]

            train_y=prepare_response_variable(train_target)
            test_y=prepare_response_variable(test_target)
            #data정의 -> train, train_y, validation,  test_y

            result = c_index_kernel_type(train, train_y, validation, test_y, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = 'ensemble_cox')
            cindex.append(result[1])

        #best_cindex : 선택된 변수별 모든 validation cindex값을 list로 저장
        best_cindex.append(np.mean(cindex))

    #max_num : best cindex의 리스트 번호
    max_num = best_cindex.index(max(best_cindex))

    #train_column : best cindex의 선택된 변수
    train_column = column_groups[max_num].columns
    
    remaining_variable_cox = remaining_variable_cox.append({"variable": list(train_column)}, ignore_index = True)

c:\Python\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Python\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Python\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\3501034185.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  remaining_variable_cox = remaining_variable_cox.append({"variable": list(train_column)

In [15]:
remaining_variable_cox

,variable
0,"[ph.ecog, pat.karno, Sex, Age, OS, Status]"
1,"[ph.ecog, ph.karno, pat.karno, wt.loss, Sex, A..."
2,"[ph.ecog, ph.karno, meal.cal, Sex, Age, OS, St..."
3,"[ph.karno, pat.karno, Sex, Age, OS, Status]"
4,"[ph.ecog, pat.karno, meal.cal, wt.loss, Sex, A..."
...,...
95,"[ph.karno, pat.karno, meal.cal, Sex, Age, OS, ..."
96,"[ph.ecog, meal.cal, wt.loss, Sex, Age, OS, Sta..."
97,"[ph.karno, pat.karno, meal.cal, wt.loss, Sex, ..."
98,"[ph.ecog, ph.karno, pat.karno, wt.loss, Sex, A..."


In [16]:
remaining_variable_cox.to_csv("lung_cox_remaining_variables.csv", index = False, encoding = 'cp949')

Kernel 종류별 결과 확인

Ensemble Cox Kernel

In [17]:
ensemble_cox_results=pd.DataFrame()

ensemble_cox_results['train_C_index']=[]
ensemble_cox_results['test_C_index']=[]

In [24]:
kernel_type = 'ensemble_cox'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_cox_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_cox_results.to_csv("lung_cox_cox.csv", index = False, encoding = 'cp949')

C:\Users\admin\AppData\Local\Temp\ipykernel_16948\4280741229.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\4280741229.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\4280741229.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admi

In [26]:
ensemble_cox_results

,train_C_index,test_C_index
0,0.667236,0.642253
1,0.665030,0.619510
2,0.633605,0.649687
3,0.699944,0.555761
4,0.668707,0.651726
...,...,...
95,0.730730,0.532979
96,0.669128,0.602331
97,0.656720,0.608233
98,0.664586,0.617922


Linear kernel

In [27]:
linear_results=pd.DataFrame()

linear_results['train_C_index']=[]
linear_results['test_C_index']=[]

In [28]:
kernel_type = 'linear'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_cox_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

linear_results.to_csv("lung_cox_linear.csv", index = False, encoding = 'cp949')


C:\Users\admin\AppData\Local\Temp\ipykernel_16948\527461989.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\527461989.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\527461989.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\52

In [29]:
linear_results

,train_C_index,test_C_index
0,0.579577,0.576803
1,0.608458,0.585761
2,0.517437,0.556747
3,0.622961,0.551596
4,0.552306,0.489467
...,...,...
95,0.553377,0.513502
96,0.528540,0.518881
97,0.543147,0.519889
98,0.601362,0.603656


Clinical Kernel

In [30]:
clinical_results=pd.DataFrame()

clinical_results['train_C_index']=[]
clinical_results['test_C_index']=[]

In [31]:
kernel_type = 'clinical'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_cox_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

clinical_results.to_csv("lung_cox_clinical.csv", index = False, encoding = 'cp949')

C:\Users\admin\AppData\Local\Temp\ipykernel_16948\1420628784.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\1420628784.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\1420628784.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipy

In [32]:
clinical_results

,train_C_index,test_C_index
0,0.667350,0.648486
1,0.671279,0.625982
2,0.645494,0.642538
3,0.692235,0.571032
4,0.673127,0.640520
...,...,...
95,0.706173,0.551571
96,0.687562,0.591142
97,0.666964,0.623034
98,0.664132,0.641551


Ensemble AFT Kernel

In [33]:
ensemble_aft_results=pd.DataFrame()

ensemble_aft_results['train_C_index']=[]
ensemble_aft_results['test_C_index']=[]

In [34]:
kernel_type = 'ensemble_aft'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_cox_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_aft_results.to_csv("lung_cox_aft.csv", index = False, encoding = 'cp949')

C:\Users\admin\AppData\Local\Temp\ipykernel_16948\419674333.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\419674333.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16948\419674333.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\admin\A

In [35]:
ensemble_aft_results

,train_C_index,test_C_index
0,0.667577,0.642253
1,0.664807,0.620897
2,0.633605,0.649687
3,0.697151,0.555761
4,0.668820,0.651726
...,...,...
95,0.750966,0.532979
96,0.669017,0.601865
97,0.656942,0.605920
98,0.672304,0.625947


In [36]:
linear_results = pd.read_csv('lung_cox_linear.csv')
clinical_results = pd.read_csv('lung_cox_clinical.csv')
ensemble_cox_results = pd.read_csv('lung_cox_cox.csv')
ensemble_aft_results = pd.read_csv('lung_cox_aft.csv')

In [38]:
remaining_variables_all = []
for i in range(0,100):
    #remaining_variables_all += eval(remaining_variable[i])
    remaining_variables_all += eval(pd.read_csv("lung_cox_remaining_variables.csv")['variable'][i])

element_counts = {}

for element in remaining_variables_all:
    if element in element_counts:
        element_counts[element] += 1
    else:
        element_counts[element] = 1

print(element_counts)

{'ph.ecog': 74, 'pat.karno': 81, 'Sex': 100, 'Age': 100, 'OS': 100, 'Status': 100, 'ph.karno': 58, 'wt.loss': 54, 'meal.cal': 51}


In [39]:
print(round(np.mean(linear_results['train_C_index']),4))
print(round(np.std(linear_results['train_C_index']),4))
print(round(np.mean(linear_results['test_C_index']),4))
print(round(np.std(linear_results['test_C_index']),4))

0.5653
0.0415
0.5559
0.0455


In [40]:
print(round(np.mean(clinical_results['train_C_index']),4))
print(round(np.std(clinical_results['train_C_index']),4))
print(round(np.mean(clinical_results['test_C_index']),4))
print(round(np.std(clinical_results['test_C_index']),4))

0.667
0.0208
0.6316
0.0341


In [41]:
print(round(np.mean(ensemble_cox_results['train_C_index']),4))
print(round(np.std(ensemble_cox_results['train_C_index']),4))
print(round(np.mean(ensemble_cox_results['test_C_index']),4))
print(round(np.std(ensemble_cox_results['test_C_index']),4))

0.6612
0.0197
0.6292
0.0348


In [42]:
print(round(np.mean(ensemble_aft_results['train_C_index']),4))
print(round(np.std(ensemble_aft_results['train_C_index']),4))
print(round(np.mean(ensemble_aft_results['test_C_index']),4))
print(round(np.std(ensemble_aft_results['test_C_index']),4))

0.6622
0.021
0.6282
0.0347


### AFT- cross validation

In [43]:
from kernel_function import ensemble_AFT_kernel, ensemble_cox_kernel, c_index_kernel_type

Best c-index를 도출하는 변수 뽑기

In [44]:
remaining_variable_aft=pd.DataFrame()

remaining_variable_aft['variable']=[]

In [ ]:
for i in range(100):
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    #os, status 분리
    drop = x_train[['Sex', 'Age', 'OS','Status']]
    x_train_drop = x_train.drop(columns=['Sex', 'Age', 'OS','Status'])

    # 데이터프레임의 열 이름을 리스트로 저장
    columns = x_train_drop.columns

    column_groups = []

    for i in range(len(columns)):
        # i+1개씩 선택한 모든 열 조합을 생성
        all_column_combinations = list(combinations(columns, i+1))

        # 각 조합에 대한 데이터프레임을 생성하고 출력
        for column_combination in all_column_combinations:
            selected_columns = list(column_combination)
            sub_train1 = x_train_drop[selected_columns]
            sub_train2 = pd.concat([sub_train1, drop], axis=1)
            column_groups.append(sub_train2)
    
    num_groups = 3
    best_cindex = []

    for i in range(len(column_groups)):
        x_groups = []
        train = column_groups[i].sample(frac=1).reset_index(drop=True)

        # train data를 그룹으로 나누기
        for i in range(num_groups):
            censored = train[train['Status'] == 0]  # 0인 행만 추출
            uncensored = train[train['Status'] == 1]  # 1인 행만 추출

            group_size1 = len(censored) // num_groups
            group_size2 = len(uncensored) // num_groups

            if i < num_groups - 1:
                #censoring 비율 맞추기
                group1 = censored.iloc[i * group_size1:(i + 1) * group_size1]
                group2 = uncensored.iloc[i * group_size2:(i + 1) * group_size2]

                group = pd.concat([group1, group2], ignore_index=True)
            else:
                group1 = censored.iloc[i * group_size1:]
                group2 = uncensored.iloc[i * group_size2:]

                group = pd.concat([group1, group2], ignore_index=True)
    
            x_groups.append(group)

        cindex = []

        for i in range(len(x_groups)):

            temp = []
            for j in range(len(x_groups)):
                if i != j:
                    temp.append(x_groups[j])
            #train = train data(등분한 것 중 1개를 제외한 나머지)
            train=pd.concat(temp)

            train = train.reset_index(drop=True)

            validation = x_groups[i]
            
            train_target=train[['Status','OS']]
            test_target=validation[['Status','OS']]

            train_y=prepare_response_variable(train_target)
            test_y=prepare_response_variable(test_target)
            #data정의 -> train, train_y, validation,  test_y

            result = c_index_kernel_type(train, train_y, validation, test_y, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = 'ensemble_aft')
            cindex.append(result[1])

        #best_cindex : 선택된 변수별 모든 validation cindex값을 list로 저장
        best_cindex.append(np.mean(cindex))

    #max_num : best cindex의 리스트 번호
    max_num = best_cindex.index(max(best_cindex))

    #train_column : best cindex의 선택된 변수
    train_column = column_groups[max_num].columns
    
    remaining_variable_aft = remaining_variable_aft.append({"variable": list(train_column)}, ignore_index = True)
    print(remaining_variable_aft)

In [47]:
remaining_variable_aft

,variable
0,"[ph.ecog, ph.karno, pat.karno, meal.cal, Sex, ..."
1,"[ph.ecog, ph.karno, pat.karno, Sex, Age, OS, S..."
2,"[ph.ecog, pat.karno, Sex, Age, OS, Status]"


In [ ]:
remaining_variable_aft.to_csv("lung_aft_remaining_variables.csv", index = False, encoding = 'cp949')

Kernel 종류별 결과 확인

Ensemble AFT Kernel

In [ ]:
ensemble_aft_results=pd.DataFrame()

ensemble_aft_results['train_C_index']=[]
ensemble_aft_results['test_C_index']=[]

In [ ]:
kernel_type = 'ensemble_aft'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_aft_results.to_csv("lung_aft_aft.csv", index = False, encoding = 'cp949')

Linear Kernel

In [ ]:
linear_results=pd.DataFrame()

linear_results['train_C_index']=[]
linear_results['test_C_index']=[]

In [ ]:
kernel_type = 'linear'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_aft_remaining_variables.csv")['variable'][i])
    
    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

linear_results.to_csv("lung_aft_linear.csv", index = False, encoding = 'cp949')


Clinical Kernel

In [ ]:
clinical_results=pd.DataFrame()

clinical_results['train_C_index']=[]
clinical_results['test_C_index']=[]

In [ ]:
kernel_type = 'clinical'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

clinical_results.to_csv("lung_aft_clinical.csv", index = False, encoding = 'cp949')


Ensemble Cox Kernel

In [ ]:
ensemble_cox_results=pd.DataFrame()

ensemble_cox_results['train_C_index']=[]
ensemble_cox_results['test_C_index']=[]

In [ ]:
kernel_type = 'ensemble_cox'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("lung_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_cox_results.to_csv("lung_aft_cox.csv", index = False, encoding = 'cp949')

In [ ]:
linear_results = pd.read_csv('lung_aft_linear.csv')
clinical_results = pd.read_csv('lung_aft_clinical.csv')
ensemble_cox_results = pd.read_csv('lung_aft_cox.csv')
ensemble_aft_results = pd.read_csv('lung_aft_aft.csv')

In [ ]:
remaining_variables_all = []
for i in range(0,100):
    #remaining_variables_all += eval(remaining_variable[i])
    variables = eval(pd.read_csv("lung_aft_remaining_variables.csv")['variable'][i])

element_counts = {}

for element in remaining_variables_all:
    if element in element_counts:
        element_counts[element] += 1
    else:
        element_counts[element] = 1

print(element_counts)

In [ ]:
print(round(np.mean(linear_results['train_C_index']),4))
print(round(np.std(linear_results['train_C_index']),4))
print(round(np.mean(linear_results['test_C_index']),4))
print(round(np.std(linear_results['test_C_index']),4))

In [ ]:
print(round(np.mean(clinical_results['train_C_index']),4))
print(round(np.std(clinical_results['train_C_index']),4))
print(round(np.mean(clinical_results['test_C_index']),4))
print(round(np.std(clinical_results['test_C_index']),4))

In [ ]:
print(round(np.mean(ensemble_cox_results['train_C_index']),4))
print(round(np.std(ensemble_cox_results['train_C_index']),4))
print(round(np.mean(ensemble_cox_results['test_C_index']),4))
print(round(np.std(ensemble_cox_results['test_C_index']),4))

In [ ]:
print(round(np.mean(ensemble_aft_results['train_C_index']),4))
print(round(np.std(ensemble_aft_results['train_C_index']),4))
print(round(np.mean(ensemble_aft_results['test_C_index']),4))
print(round(np.std(ensemble_aft_results['test_C_index']),4))